# 🔍 Week 07-08 · Notebook 07 · Advanced RAG Patterns

Design multi-stage retrieval pipelines that adapt to equipment type, language, and safety risk.

## 🎯 Learning Objectives
- Implement multi-query generation to cover ambiguous technician prompts.
- Use hierarchical retrieval (document → section → paragraph) tuned for SOPs.
- Introduce domain heuristics (equipment criticality, downtime cost).
- Export explainability artefacts for compliance review.

## 🧩 Scenario
When maintenance asks "Why is spindle vibration high?" the system must retrieve root-cause entries from sensors, SOPs, and prior incidents, ranking by safety priority.

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- 1. Setup: Create a sample vector store ---
# This represents our knowledge base of SOPs, maintenance logs, etc.
sample_texts = [
    "SOP-101: For high spindle vibration, first check for loose tool holders. Torque to 50 Nm.",
    "Incident-552: High vibration on Press-04 was caused by bearing failure. Downtime: 8 hours. Cost: $50,000.",
    "SOP-102: Spindle bearing replacement must be done in a clean environment. Use only approved lubricants.",
    "Safety-Alert-7: Immediate shutdown is required if spindle vibration exceeds 10 mm/s. This is a critical safety issue.",
    "Troubleshooting-Guide: Common causes of vibration include tool imbalance, bearing wear, and incorrect speed settings."
]
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectorstore = Chroma.from_texts(sample_texts, embeddings)

# --- 2. Multi-Query Retriever ---
# This retriever generates multiple variations of the user's question to improve recall.
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), 
    llm=llm
)

# --- 3. Build the RAG Chain using LCEL ---
# This is the modern way to build RAG chains in LangChain.
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful manufacturing assistant. Answer the user's question based on the following context. Prioritize safety alerts."),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# --- 4. Run the chain ---
question = 'Why is spindle vibration high after maintenance? Provide troubleshooting steps.'
response = rag_chain.invoke(question)

print("--- RAG Response ---")
print(response)

# --- Bonus: See the generated queries ---
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

retriever.get_relevant_documents(question)

### 🧭 Hierarchical Retrieval
1. Retrieve relevant SOP sections via embeddings.
2. Drill down to paragraphs referencing maintenance step IDs.
3. Apply heuristic scoring: safety incidents > downtime costs > general tips.
4. Log retrieval path for explainability.

In [ ]:
def rank_documents(docs):
    """
    Applies a simple heuristic to rank documents based on keywords.
    In a real system, this would be more sophisticated, using metadata, etc.
    """
    ranked_docs = []
    for doc in docs:
        content = doc.page_content.lower()
        score = 0
        if "safety-alert" in content or "critical safety" in content:
            score = 100
        elif "incident" in content or "downtime" in content:
            score = 50
        elif "sop" in content:
            score = 20
        else:
            score = 10
        
        ranked_docs.append({"doc": doc, "score": score})
        
    # Sort documents by score in descending order
    return sorted(ranked_docs, key=lambda x: x['score'], reverse=True)

# --- Example Usage ---
# First, retrieve the documents
retrieved_docs = retriever.get_relevant_documents(question)

# Then, rank them
ranked_results = rank_documents(retrieved_docs)

print("\n--- Heuristically Ranked Documents ---")
for result in ranked_results:
    print(f"Score: {result['score']}, Content: {result['doc'].page_content[:80]}...")

## 🧪 Lab Assignment
1. Implement hierarchical retriever using LangChain `ParentDocumentRetriever` for SOP tree.
2. Add weight adjustments based on downtime impact from cost database.
3. Export retrieval traces to JSON for auditors.
4. Present RAG improvements vs. baseline to stakeholders.

## ✅ Checklist
- [ ] Multi-query retriever configured
- [ ] Hierarchical retrieval implemented
- [ ] Heuristic ranking documented
- [ ] Lab deliverables shared

## 📚 References
- LangChain Multi-Query Retriever
- Hierarchical Retrieval Patterns
- Week 09 Evaluation Harness